# Importations et nettoyage

In [10]:
import pandas as pd


matrice_temps = pd.read_csv(r'C:\Users\Nicolas et Maëva\Desktop\Projets DataSpell\DataBird\Projet_final\matrice_temps_complete.csv')
code_espelia_code_insee = pd.read_csv(r'C:\Users\Nicolas et Maëva\Desktop\Projets DataSpell\DataBird\Projet_final\Coordonnées_villes_par_codes_INSEE.csv')
table_insee = pd.read_csv(r'C:\Users\Nicolas et Maëva\Desktop\Projets DataSpell\DataBird\Projet_final\table_insee.csv')
infrastructures = pd.read_csv(r'C:\Users\Nicolas et Maëva\Desktop\Projets DataSpell\DataBird\Projet_final\table_infrastructures.csv')
table_recensement = pd.read_csv(r'C:\Users\Nicolas et Maëva\Desktop\Projets DataSpell\DataBird\Projet_final\table_recensement.csv')

matrice_temps['Temps_trajet'] = matrice_temps['Temps_trajet'].astype(int)
matrice_temps['Commune_destination'] = matrice_temps['Commune_destination'].astype(str)
matrice_temps['Commune_depart'] = matrice_temps['Commune_depart'].astype(str)

code_espelia_code_insee['Index'] = code_espelia_code_insee['Index'].astype(str)

table_reference = code_espelia_code_insee.merge(table_insee, left_on='INSEE_COM', right_on='CODGEO')
table_reference.drop(columns=['INSEE_COM', 'Unnamed: 0', ], inplace=True)
table_reference.rename(columns={'Index':'Index_espelia', 'CODGEO':'CODGEO_INSEE'}, inplace=True)

# Implémentation de la fonction de référencement de codes INSEE / ESPELIA

In [11]:
def codes(nom_de_ville) :
    row = table_reference[table_reference['LIBGEO'] == str(nom_de_ville)]
    return row

# Implémentations des différents taux de captation

In [12]:
def colonne_conditionnelle (ligne):
    if ligne['zone'] == 'Z1':
        return 100
    elif ligne['zone'] == 'Z2':
        return 80
    elif ligne['zone'] == 'Z3':
        return 40
    elif ligne['zone'] == 'Z4':
        return 20

# Implémentation de la fonction principale de création de zones de captation et de vérification de la présence d'un crématorium plus proche que notre projet

In [13]:
def zonage(code_espelia) :

    # Définition des zones de captation et création de la colonne étiquette 'zone' pour chaque zone

    Z1 = (matrice_temps[(matrice_temps['Commune_depart'] == str(code_espelia))
                        & (matrice_temps['Temps_trajet'] < 900.0)]) \
        .merge(code_espelia_code_insee, left_on='Commune_destination', right_on='Index') \
        .merge(table_insee, left_on='INSEE_COM', right_on='CODGEO') \
        .merge(table_recensement, on='CODGEO') \
        .sort_values(by='Temps_trajet') \
        .drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'INSEE_COM', 'Unnamed: 0', 'Commune_depart', 'Commune_destination'])
    Z1['zone'] = 'Z1'

    Z2 = (matrice_temps[(matrice_temps['Commune_depart'] == str(code_espelia))
                        & (matrice_temps['Temps_trajet'] >= 900.0)
                        & (matrice_temps['Temps_trajet'] < 1800.0)]) \
        .merge(code_espelia_code_insee, left_on='Commune_destination', right_on='Index') \
        .merge(table_insee, left_on='INSEE_COM', right_on='CODGEO') \
        .merge(table_recensement, on='CODGEO') \
        .sort_values(by='Temps_trajet') \
        .drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'INSEE_COM', 'Unnamed: 0', 'Commune_depart', 'Commune_destination'])
    Z2['zone'] = 'Z2'

    Z3 = (matrice_temps[(matrice_temps['Commune_depart'] == str(code_espelia))
                        & (matrice_temps['Temps_trajet'] >= 1800.0)
                        & (matrice_temps['Temps_trajet'] < 2700.0)]) \
        .merge(code_espelia_code_insee, left_on='Commune_destination', right_on='Index') \
        .merge(table_insee, left_on='INSEE_COM', right_on='CODGEO') \
        .merge(table_recensement, on='CODGEO') \
        .sort_values(by='Temps_trajet') \
        .drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'INSEE_COM', 'Unnamed: 0', 'Commune_depart', 'Commune_destination'])
    Z3['zone'] = 'Z3'

    Z4 = (matrice_temps[(matrice_temps['Commune_depart'] == str(code_espelia))
                        & (matrice_temps['Temps_trajet'] >= 2700.0)]) \
        .merge(code_espelia_code_insee, left_on='Commune_destination', right_on='Index') \
        .merge(table_insee, left_on='INSEE_COM', right_on='CODGEO') \
        .merge(table_recensement, on='CODGEO') \
        .sort_values(by='Temps_trajet') \
        .drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'INSEE_COM', 'Unnamed: 0', 'Commune_depart', 'Commune_destination'])
    Z4['zone'] = 'Z4'

    # Concaténation des zones de captation et création des colonnes 'Taux_captation' et 'Population_captée' 
    # grâce à notre fonction colonne_conditionnelle

    zonage_complet = pd.concat([Z1, Z2, Z3, Z4])
    zonage_complet['Taux_captation'] = zonage_complet.apply(colonne_conditionnelle, axis=1)
    zonage_complet['Population_captée'] = (zonage_complet['PMUN20'] * zonage_complet['Taux_captation']/100).astype(int)


    # Création de la liste des villes à vérifier

    villes_a_verif = zonage_complet.iloc[1:, :] \
        .set_index('Index')['Temps_trajet'] \
        .to_dict()


    # Pour chaque ville, on vérifie si notre projet de crématorium serait le plus proche de la ville,
    # si c'est le cas, on garde la ville, sinon on l'exclut du zonage final

    progression = 0
    villes_a_garder = []
    villes_a_garder.append(code_espelia)
    infrastructures['STATUT'].replace({'Existant':1, 'Projet':0}, inplace=True)
    for ville in villes_a_verif.keys():
        temps_a_comparer = villes_a_verif[ville]
        df = (matrice_temps[matrice_temps['Commune_depart'] == ville]) \
            .merge(code_espelia_code_insee, left_on='Commune_destination', right_on='Index') \
            .merge(infrastructures, left_on='INSEE_COM', right_on='CODGEO') \
            .sort_values(by='Temps_trajet')
        filtered_df = df.loc[df['STATUT'] == 1, 'Temps_trajet']
        if not filtered_df.empty:
            temps_crematorium_plus_proche = filtered_df.iloc[0]
        else:
            temps_crematorium_plus_proche = 99999999
        progression += 1
        print(f"Progression : {progression} / {len(villes_a_verif)}")
        if temps_a_comparer < temps_crematorium_plus_proche:
            villes_a_garder.append(ville)
    zonage_complet = zonage_complet[zonage_complet['Index'].isin(villes_a_garder)]

    return zonage_complet

# Utilisation

In [20]:
zonage_carcassonne = zonage((codes('Carcassonne'))['Index_espelia'].iloc[0])
zonage_carcassonne

Progression : 1 / 1168
Progression : 2 / 1168
Progression : 3 / 1168
Progression : 4 / 1168
Progression : 5 / 1168
Progression : 6 / 1168
Progression : 7 / 1168
Progression : 8 / 1168
Progression : 9 / 1168
Progression : 10 / 1168
Progression : 11 / 1168
Progression : 12 / 1168
Progression : 13 / 1168
Progression : 14 / 1168
Progression : 15 / 1168
Progression : 16 / 1168
Progression : 17 / 1168
Progression : 18 / 1168
Progression : 19 / 1168
Progression : 20 / 1168
Progression : 21 / 1168
Progression : 22 / 1168
Progression : 23 / 1168
Progression : 24 / 1168
Progression : 25 / 1168
Progression : 26 / 1168
Progression : 27 / 1168
Progression : 28 / 1168
Progression : 29 / 1168
Progression : 30 / 1168
Progression : 31 / 1168
Progression : 32 / 1168
Progression : 33 / 1168
Progression : 34 / 1168
Progression : 35 / 1168
Progression : 36 / 1168
Progression : 37 / 1168
Progression : 38 / 1168
Progression : 39 / 1168
Progression : 40 / 1168
Progression : 41 / 1168
Progression : 42 / 1168
P

,Temps_trajet,Index,CentreLong,CentreLat,CODGEO,LIBGEO,DEP,REG,PMUN20,zone,Taux_captation,Population_captée
10,0,9271,"2,35289992062672","43,2120619561765",11069,Carcassonne,11,76,46673,Z1,100,46673
25,369,27875,"2,36393639081778","43,1850348035394",11088,Cazilhac,11,76,1640,Z1,100,1640
20,505,21347,"2,3193292766512","43,2442104755377",11279,Pennautier,11,76,2714,Z1,100,2714
15,542,15947,"2,32709796822045","43,1681713149989",11085,Cavanac,11,76,960,Z1,100,960
6,551,7894,"2,38732066725991","43,1739892338817",11272,Palaja,11,76,2496,Z1,100,2496
...,...,...,...,...,...,...,...,...,...,...,...,...
245,5118,9453,"2,12573854532552","42,788624593138",11038,Bessède-de-Sault,11,76,46,Z4,20,9
750,5185,29437,"2,16799557323596","42,7855231367332",11093,Le Clat,11,76,28,Z4,20,5
729,5244,28438,"2,06887586773508","42,7302930695826",09252,Rouze,09,76,90,Z4,20,18
344,5253,13168,"2,08200384752841","42,7685488294282",11147,Fontanès-de-Sault,11,76,5,Z4,20,1


In [21]:
zonage_carcassonne.to_csv(r'zonage_carcassonne')